# Production LlamaIndex Patterns with AgentCore Browser Tool

This notebook demonstrates production-ready patterns for deploying LlamaIndex with Amazon Bedrock AgentCore Browser Tool, focusing on:

- **Scalable Architecture**: Session pooling, resource management, auto-scaling
- **Monitoring & Observability**: Built-in AgentCore monitoring for sensitive operations
- **Error Handling & Recovery**: Robust patterns protecting sensitive data
- **Compliance & Audit Logging**: Enterprise-grade logging and compliance

## Prerequisites
- AWS credentials configured
- Completed previous notebooks in this series
- Understanding of production deployment patterns

In [ ]:
import os
import json
import asyncio
import logging
import time
import uuid
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Any
from dataclasses import dataclass, asdict
from concurrent.futures import ThreadPoolExecutor
from contextlib import asynccontextmanager

# LlamaIndex imports
from llama_index.core import VectorStoreIndex, Document, Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding

# AgentCore Browser Tool
from bedrock_agentcore.tools.browser_client import BrowserSession
from bedrock_agentcore.tools.browser_client.browser_session import BrowserSessionConfig

# Production utilities
from examples.secure_rag_pipeline import SecureRAGPipeline
from examples.agentcore_browser_loader import AgentCoreBrowserLoader
from examples.sensitive_data_handler import SensitiveDataHandler

# Configure production logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('production_llamaindex_agentcore.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

print("✅ Production dependencies loaded")

## 1. Production Configuration & Session Management

In [ ]:
@dataclass
class ProductionConfig:
    """Production configuration for LlamaIndex-AgentCore integration."""
    environment: str = "production"
    region: str = "us-east-1"
    max_concurrent_sessions: int = 10
    session_timeout_minutes: int = 30
    enable_audit_logging: bool = True
    enable_pii_detection: bool = True
    enable_metrics: bool = True
    max_retries: int = 3
    chunk_size: int = 1024
    max_tokens: int = 4096

class ProductionSessionPool:
    """Production session pool with monitoring and auto-cleanup."""
    
    def __init__(self, config: ProductionConfig):
        self.config = config
        self.active_sessions: Dict[str, Dict] = {}
        self.metrics = {
            'total_created': 0,
            'current_active': 0,
            'peak_concurrent': 0
        }
    
    async def get_session(self, session_id: Optional[str] = None) -> BrowserSession:
        """Get or create monitored browser session."""
        if session_id and session_id in self.active_sessions:
            session_info = self.active_sessions[session_id]
            if self._is_session_valid(session_info):
                logger.info(f"Reusing session: {session_id}")
                return session_info['session']
            else:
                await self._cleanup_session(session_id)
        
        # Check session limits
        if len(self.active_sessions) >= self.config.max_concurrent_sessions:
            await self._cleanup_oldest_session()
        
        # Create new session
        session_id = session_id or str(uuid.uuid4())
        session = await self._create_monitored_session(session_id)
        
        # Update metrics
        self.metrics['total_created'] += 1
        self.metrics['current_active'] = len(self.active_sessions)
        self.metrics['peak_concurrent'] = max(
            self.metrics['peak_concurrent'],
            self.metrics['current_active']
        )
        
        logger.info(f"Created session: {session_id}")
        return session
    
    async def _create_monitored_session(self, session_id: str) -> BrowserSession:
        """Create session with production monitoring."""
        session_config = BrowserSessionConfig(
            region=self.config.region,
            enable_observability=True,
            enable_audit_logging=self.config.enable_audit_logging,
            session_timeout=self.config.session_timeout_minutes * 60
        )
        
        session = BrowserSession(config=session_config)
        await session.start()
        
        self.active_sessions[session_id] = {
            'session': session,
            'created_at': datetime.utcnow(),
            'last_used': datetime.utcnow(),
            'operation_count': 0
        }
        
        return session
    
    def _is_session_valid(self, session_info: Dict) -> bool:
        """Check session validity."""
        timeout = timedelta(minutes=self.config.session_timeout_minutes)
        return datetime.utcnow() - session_info['last_used'] < timeout
    
    async def _cleanup_session(self, session_id: str):
        """Clean up specific session."""
        if session_id in self.active_sessions:
            session_info = self.active_sessions[session_id]
            try:
                await session_info['session'].close()
            except Exception as e:
                logger.warning(f"Error closing session {session_id}: {e}")
            
            del self.active_sessions[session_id]
            self.metrics['current_active'] = len(self.active_sessions)
            logger.info(f"Cleaned up session: {session_id}")
    
    async def _cleanup_oldest_session(self):
        """Clean up oldest session."""
        if not self.active_sessions:
            return
        
        oldest_id = min(
            self.active_sessions.keys(),
            key=lambda x: self.active_sessions[x]['created_at']
        )
        await self._cleanup_session(oldest_id)
    
    async def shutdown(self):
        """Shutdown all sessions."""
        for session_id in list(self.active_sessions.keys()):
            await self._cleanup_session(session_id)
        logger.info("Session pool shutdown complete")

# Initialize production components
config = ProductionConfig()
session_pool = ProductionSessionPool(config)
print(f"✅ Production config initialized: {config.environment}")
print(f"   Max sessions: {config.max_concurrent_sessions}")
print(f"   Audit logging: {config.enable_audit_logging}")

## 2. Production Monitoring & Error Handling

In [ ]:
class ProductionMonitor:
    """Production monitoring for LlamaIndex-AgentCore operations."""
    
    def __init__(self, config: ProductionConfig):
        self.config = config
        self.metrics = {
            'operations': {'total': 0, 'successful': 0, 'failed': 0, 'sensitive': 0},
            'performance': {'avg_response_time': 0.0, 'total_data_mb': 0.0},
            'security': {'pii_detections': 0, 'masking_ops': 0, 'violations': 0},
            'errors': {'session_errors': 0, 'auth_errors': 0, 'processing_errors': 0}
        }
        self.audit_log = []
        self.start_time = datetime.utcnow()
    
    def record_operation(self, operation_type: str, success: bool, duration: float, 
                        data_size_mb: float = 0.0, sensitive_data: bool = False, metadata: Dict = None):
        """Record operation with comprehensive metrics."""
        self.metrics['operations']['total'] += 1
        
        if success:
            self.metrics['operations']['successful'] += 1
        else:
            self.metrics['operations']['failed'] += 1
        
        if sensitive_data:
            self.metrics['operations']['sensitive'] += 1
        
        # Update performance metrics
        total_ops = self.metrics['operations']['total']
        current_avg = self.metrics['performance']['avg_response_time']
        self.metrics['performance']['avg_response_time'] = (
            (current_avg * (total_ops - 1) + duration) / total_ops
        )
        self.metrics['performance']['total_data_mb'] += data_size_mb
        
        # Create audit entry
        if self.config.enable_audit_logging:
            audit_entry = {
                'timestamp': datetime.utcnow().isoformat(),
                'operation_type': operation_type,
                'success': success,
                'duration_seconds': duration,
                'data_size_mb': data_size_mb,
                'sensitive_data': sensitive_data,
                'metadata': metadata or {}
            }
            self.audit_log.append(audit_entry)
            logger.info(f"Operation recorded: {operation_type} - Success: {success}")
    
    def record_security_event(self, event_type: str, details: Dict = None):
        """Record security events."""
        if event_type == 'pii_detection':
            self.metrics['security']['pii_detections'] += 1
        elif event_type == 'data_masking':
            self.metrics['security']['masking_ops'] += 1
        elif event_type == 'security_violation':
            self.metrics['security']['violations'] += 1
        
        if self.config.enable_audit_logging:
            security_entry = {
                'timestamp': datetime.utcnow().isoformat(),
                'event_type': event_type,
                'details': details or {},
                'severity': 'HIGH' if event_type == 'security_violation' else 'INFO'
            }
            self.audit_log.append(security_entry)
            logger.warning(f"Security event: {event_type}")
    
    def get_health_status(self) -> Dict[str, Any]:
        """Get system health status."""
        total_ops = self.metrics['operations']['total']
        success_rate = (
            self.metrics['operations']['successful'] / total_ops * 100
            if total_ops > 0 else 100.0
        )
        
        uptime = datetime.utcnow() - self.start_time
        
        return {
            'status': 'healthy' if success_rate > 95 else 'degraded' if success_rate > 80 else 'unhealthy',
            'uptime_seconds': uptime.total_seconds(),
            'success_rate_percent': success_rate,
            'total_operations': total_ops,
            'avg_response_time': self.metrics['performance']['avg_response_time'],
            'security_violations': self.metrics['security']['violations']
        }
    
    def export_audit_log(self, filename: str = None) -> str:
        """Export audit log for compliance."""
        if not filename:
            timestamp = datetime.utcnow().strftime('%Y%m%d_%H%M%S')
            filename = f'audit_log_{timestamp}.json'
        
        with open(filename, 'w') as f:
            json.dump({
                'export_timestamp': datetime.utcnow().isoformat(),
                'total_entries': len(self.audit_log),
                'audit_entries': self.audit_log
            }, f, indent=2)
        
        logger.info(f"Audit log exported to {filename}")
        return filename

class ProductionErrorHandler:
    """Production error handling with circuit breaker pattern."""
    
    def __init__(self, config: ProductionConfig, monitor: ProductionMonitor):
        self.config = config
        self.monitor = monitor
        self.circuit_breaker_state = {}
    
    async def handle_with_retry(self, operation_func, operation_name: str, *args, **kwargs):
        """Execute operation with retry logic and monitoring."""
        last_error = None
        
        for attempt in range(self.config.max_retries + 1):
            try:
                start_time = time.time()
                result = await operation_func(*args, **kwargs)
                duration = time.time() - start_time
                
                # Record successful operation
                self.monitor.record_operation(
                    operation_type=operation_name,
                    success=True,
                    duration=duration,
                    metadata={'attempt': attempt + 1}
                )
                
                return result
                
            except Exception as e:
                last_error = e
                duration = time.time() - start_time
                
                # Record failed operation
                self.monitor.record_operation(
                    operation_type=operation_name,
                    success=False,
                    duration=duration,
                    metadata={'attempt': attempt + 1, 'error': str(e)}
                )
                
                if attempt < self.config.max_retries:
                    # Wait before retry with exponential backoff
                    wait_time = 2 ** attempt
                    logger.warning(f"Attempt {attempt + 1} failed for {operation_name}, retrying in {wait_time}s")
                    await asyncio.sleep(wait_time)
                else:
                    logger.error(f"All {self.config.max_retries + 1} attempts failed for {operation_name}")
        
        raise last_error

# Initialize monitoring and error handling
monitor = ProductionMonitor(config)
error_handler = ProductionErrorHandler(config, monitor)
print("✅ Production monitoring initialized")
print(f"   Audit logging: {config.enable_audit_logging}")
print(f"   Max retries: {config.max_retries}")

## 3. Production RAG Pipeline with Security

In [ ]:
class ProductionLlamaIndexRAG:
    """Production-ready LlamaIndex RAG with AgentCore Browser Tool."""
    
    def __init__(self, config: ProductionConfig, monitor: ProductionMonitor,
                 error_handler: ProductionErrorHandler, session_pool: ProductionSessionPool):
        self.config = config
        self.monitor = monitor
        self.error_handler = error_handler
        self.session_pool = session_pool
        
        # Initialize LlamaIndex components
        self.llm = Bedrock(
            model="anthropic.claude-3-sonnet-20240229-v1:0",
            region_name=config.region,
            max_tokens=config.max_tokens
        )
        
        self.embed_model = BedrockEmbedding(
            model="amazon.titan-embed-text-v1",
            region_name=config.region
        )
        
        Settings.llm = self.llm
        Settings.embed_model = self.embed_model
        Settings.chunk_size = config.chunk_size
        
        # Initialize secure components
        self.browser_loader = AgentCoreBrowserLoader(region=config.region)
        self.data_handler = SensitiveDataHandler()
        self.secure_rag = SecureRAGPipeline()
        
        self.index = None
        self.query_engine = None
    
    async def ingest_secure_web_data(self, urls: List[str], 
                                   auth_configs: Dict[str, Dict] = None) -> Dict[str, Any]:
        """Ingest web data securely with monitoring."""
        async def _ingest_operation():
            session = await self.session_pool.get_session()
            documents = []
            stats = {
                'total_urls': len(urls),
                'successful': 0,
                'failed': 0,
                'sensitive_docs': 0,
                'total_data_mb': 0.0
            }
            
            try:
                for url in urls:
                    try:
                        logger.info(f"Processing URL: {url}")
                        
                        # Get auth config if provided
                        auth_config = auth_configs.get(url) if auth_configs else None
                        
                        # Load web content securely
                        web_documents = await self.browser_loader.load_with_session(
                            session, url, auth_config
                        )
                        
                        # Process documents for sensitive data
                        for doc in web_documents:
                            # Detect sensitive data
                            sensitivity_analysis = self.data_handler.analyze_sensitivity(doc.text)
                            
                            if sensitivity_analysis['has_sensitive_data']:
                                stats['sensitive_docs'] += 1
                                self.monitor.record_security_event('pii_detection', {
                                    'url': url,
                                    'pii_types': sensitivity_analysis['pii_types']
                                })
                                
                                # Apply data masking
                                masked_doc = self.data_handler.mask_sensitive_data(doc)
                                documents.append(masked_doc)
                                
                                self.monitor.record_security_event('data_masking', {
                                    'url': url,
                                    'masking_applied': True
                                })
                            else:
                                documents.append(doc)
                            
                            # Update data size
                            doc_size_mb = len(doc.text.encode('utf-8')) / (1024 * 1024)
                            stats['total_data_mb'] += doc_size_mb
                        
                        stats['successful'] += 1
                        logger.info(f"Successfully processed {len(web_documents)} documents from {url}")
                        
                    except Exception as e:
                        stats['failed'] += 1
                        logger.error(f"Failed to process URL {url}: {e}")
                        continue
                
                # Create secure vector index
                if documents:
                    self.index = await self._create_secure_index(documents)
                    self.query_engine = self.index.as_query_engine(
                        response_mode="compact",
                        streaming=False
                    )
                    logger.info(f"Created secure index with {len(documents)} documents")
                
                return {
                    'success': True,
                    'documents_processed': len(documents),
                    'statistics': stats,
                    'index_created': self.index is not None
                }
                
            finally:
                if session:
                    await session.close()
        
        return await self.error_handler.handle_with_retry(
            _ingest_operation, "secure_web_ingestion"
        )
    
    async def _create_secure_index(self, documents: List[Document]) -> VectorStoreIndex:
        """Create secure vector index with monitoring."""
        try:
            # Parse documents with security-aware chunking
            parser = SentenceSplitter(
                chunk_size=self.config.chunk_size,
                chunk_overlap=20
            )
            
            nodes = parser.get_nodes_from_documents(documents)
            index = VectorStoreIndex(nodes)
            
            logger.info(f"Created secure vector index with {len(nodes)} nodes")
            return index
            
        except Exception as e:
            logger.error(f"Failed to create secure index: {e}")
            raise
    
    async def query_with_security(self, query: str) -> Dict[str, Any]:
        """Execute query with security and monitoring."""
        async def _query_operation():
            if not self.query_engine:
                raise ValueError("No index available. Please ingest data first.")
            
            # Sanitize query
            sanitized_query = self.data_handler.sanitize_query(query)
            
            if sanitized_query != query:
                self.monitor.record_security_event('query_sanitization', {
                    'original_length': len(query),
                    'sanitized_length': len(sanitized_query)
                })
            
            # Execute query
            start_time = time.time()
            response = self.query_engine.query(sanitized_query)
            query_duration = time.time() - start_time
            
            # Sanitize response
            sanitized_response = self.data_handler.sanitize_response(str(response))
            
            return {
                'success': True,
                'response': sanitized_response,
                'query_duration': query_duration,
                'source_nodes': len(response.source_nodes) if hasattr(response, 'source_nodes') else 0,
                'sanitization_applied': sanitized_response != str(response)
            }
        
        return await self.error_handler.handle_with_retry(
            _query_operation, "secure_query_execution"
        )
    
    def get_system_health(self) -> Dict[str, Any]:
        """Get comprehensive system health."""
        return {
            'monitor_health': self.monitor.get_health_status(),
            'session_pool_metrics': self.session_pool.metrics,
            'index_status': {
                'index_created': self.index is not None,
                'query_engine_ready': self.query_engine is not None
            }
        }
    
    async def shutdown(self):
        """Graceful shutdown."""
        logger.info("Shutting down production RAG system...")
        
        # Export audit log
        audit_file = self.monitor.export_audit_log()
        logger.info(f"Audit log exported to: {audit_file}")
        
        # Shutdown session pool
        await self.session_pool.shutdown()
        
        logger.info("Production RAG system shutdown complete")

# Initialize production RAG system
production_rag = ProductionLlamaIndexRAG(config, monitor, error_handler, session_pool)
print("✅ Production RAG system initialized")
print(f"   LLM: {production_rag.llm.model}")
print(f"   Chunk size: {config.chunk_size}")

## 4. Production Demonstration & Compliance

In [ ]:
async def demonstrate_production_patterns():
    """Demonstrate production patterns with monitoring."""
    
    print("🚀 Production LlamaIndex-AgentCore Demonstration")
    print("=" * 55)
    
    # Production URLs with different sensitivity levels
    production_urls = [
        "https://httpbin.org/forms/post",  # Form with potential PII
        "https://httpbin.org/basic-auth/user/pass",  # Authenticated content
        "https://httpbin.org/json",  # JSON data
        "https://httpbin.org/html"   # HTML content
    ]
    
    auth_configs = {
        "https://httpbin.org/basic-auth/user/pass": {
            "type": "basic",
            "username": "user",
            "password": "pass"
        }
    }
    
    try:
        # 1. Secure data ingestion
        print("\n📥 Phase 1: Secure Data Ingestion")
        print("-" * 35)
        
        ingestion_result = await production_rag.ingest_secure_web_data(
            urls=production_urls,
            auth_configs=auth_configs
        )
        
        print(f"✅ Ingestion completed:")
        print(f"   Documents processed: {ingestion_result['documents_processed']}")
        print(f"   Sensitive documents: {ingestion_result['statistics']['sensitive_docs']}")
        print(f"   Data size: {ingestion_result['statistics']['total_data_mb']:.2f} MB")
        print(f"   Success rate: {ingestion_result['statistics']['successful']}/{ingestion_result['statistics']['total_urls']}")
        
        # 2. Secure querying
        print("\n🔍 Phase 2: Secure Query Execution")
        print("-" * 35)
        
        test_queries = [
            "What information is available in the processed documents?",
            "Are there any forms or authentication methods?",
            "Summarize the JSON data structure found"
        ]
        
        for i, query in enumerate(test_queries, 1):
            print(f"\nQuery {i}: {query}")
            
            query_result = await production_rag.query_with_security(query)
            
            print(f"✅ Response ({query_result['query_duration']:.2f}s):")
            print(f"   {query_result['response'][:150]}...")
            print(f"   Source nodes: {query_result['source_nodes']}")
            print(f"   Sanitization: {query_result['sanitization_applied']}")
        
        # 3. System health monitoring
        print("\n📊 Phase 3: System Health & Metrics")
        print("-" * 35)
        
        health_status = production_rag.get_system_health()
        
        print(f"System Status: {health_status['monitor_health']['status'].upper()}")
        print(f"Success Rate: {health_status['monitor_health']['success_rate_percent']:.1f}%")
        print(f"Total Operations: {health_status['monitor_health']['total_operations']}")
        print(f"Avg Response Time: {health_status['monitor_health']['avg_response_time']:.2f}s")
        print(f"Security Violations: {health_status['monitor_health']['security_violations']}")
        
        # 4. Compliance reporting
        print("\n📋 Phase 4: Compliance Reporting")
        print("-" * 30)
        
        # Generate compliance metrics
        compliance_metrics = {
            'audit_log_entries': len(monitor.audit_log),
            'sensitive_operations': monitor.metrics['operations']['sensitive'],
            'pii_detections': monitor.metrics['security']['pii_detections'],
            'masking_operations': monitor.metrics['security']['masking_ops'],
            'data_protection_enabled': config.enable_pii_detection,
            'audit_logging_enabled': config.enable_audit_logging
        }
        
        print(f"Audit Log Entries: {compliance_metrics['audit_log_entries']}")
        print(f"Sensitive Operations: {compliance_metrics['sensitive_operations']}")
        print(f"PII Detections: {compliance_metrics['pii_detections']}")
        print(f"Masking Operations: {compliance_metrics['masking_operations']}")
        print(f"Data Protection: {'✅ Enabled' if compliance_metrics['data_protection_enabled'] else '❌ Disabled'}")
        print(f"Audit Logging: {'✅ Enabled' if compliance_metrics['audit_logging_enabled'] else '❌ Disabled'}")
        
        # Export audit log
        audit_filename = monitor.export_audit_log()
        print(f"\n📄 Audit log exported: {audit_filename}")
        
        print("\n✅ Production demonstration completed successfully!")
        
        return {
            'success': True,
            'ingestion_result': ingestion_result,
            'health_status': health_status,
            'compliance_metrics': compliance_metrics,
            'audit_file': audit_filename
        }
        
    except Exception as e:
        print(f"\n❌ Demonstration failed: {e}")
        logger.error(f"Production demonstration failed: {e}")
        return {'success': False, 'error': str(e)}

# Production best practices summary
def display_production_best_practices():
    """Display production best practices."""
    
    best_practices = {
        "🏗️ Architecture": [
            "Use session pooling for efficient resource management",
            "Implement circuit breakers for resilient error handling",
            "Design for horizontal scaling across regions",
            "Separate sensitive and non-sensitive data pipelines"
        ],
        "🔒 Security": [
            "Always enable PII detection and masking",
            "Use AgentCore's containerized isolation",
            "Implement comprehensive audit logging",
            "Never expose sensitive data in error messages",
            "Encrypt data at rest and in transit"
        ],
        "📊 Monitoring": [
            "Monitor session pool utilization",
            "Set up alerts for security violations",
            "Track PII detection and masking rates",
            "Monitor authentication success rates",
            "Implement real-time dashboards"
        ],
        "⚠️ Error Handling": [
            "Implement retry logic with exponential backoff",
            "Use circuit breakers to prevent cascade failures",
            "Handle session timeouts gracefully",
            "Sanitize all error messages",
            "Implement fallback strategies"
        ],
        "📋 Compliance": [
            "Maintain comprehensive audit trails",
            "Implement data retention policies",
            "Regular compliance reporting",
            "Document data flows and processing",
            "Conduct security assessments"
        ]
    }
    
    print("\n📚 Production Best Practices")
    print("=" * 35)
    
    for category, practices in best_practices.items():
        print(f"\n{category}")
        for i, practice in enumerate(practices, 1):
            print(f"  {i}. {practice}")

# Run demonstration
print("Starting production demonstration...")
demo_result = await demonstrate_production_patterns()

if demo_result['success']:
    print("\n🎉 Production patterns demonstration completed!")
    display_production_best_practices()
else:
    print(f"\n⚠️ Issues encountered: {demo_result.get('error', 'Unknown')}")

## 5. Cleanup & Summary

In [ ]:
async def production_cleanup():
    """Production cleanup and final reporting."""
    
    print("\n🧹 Production Cleanup")
    print("=" * 25)
    
    try:
        # Final metrics
        final_health = production_rag.get_system_health()
        
        print("\n📈 Final System Metrics:")
        print(f"   Total Operations: {final_health['monitor_health']['total_operations']}")
        print(f"   Success Rate: {final_health['monitor_health']['success_rate_percent']:.1f}%")
        print(f"   Avg Response Time: {final_health['monitor_health']['avg_response_time']:.2f}s")
        print(f"   Security Violations: {final_health['monitor_health']['security_violations']}")
        
        # Export final audit log
        final_audit = monitor.export_audit_log('final_audit_log.json')
        print(f"\n📄 Final audit log: {final_audit}")
        
        # Shutdown systems
        await production_rag.shutdown()
        
        print("\n✅ Production cleanup completed successfully")
        
        return {'cleanup_success': True}
        
    except Exception as e:
        print(f"\n❌ Cleanup error: {e}")
        return {'cleanup_success': False, 'error': str(e)}

# Perform cleanup
cleanup_result = await production_cleanup()

print("\n" + "=" * 60)
print("📋 PRODUCTION TUTORIAL SUMMARY")
print("=" * 60)
print("\n✅ What We Accomplished:")
print("   • Scalable session pooling with auto-cleanup")
print("   • Comprehensive monitoring and observability")
print("   • Robust error handling with circuit breakers")
print("   • Security-first design with PII protection")
print("   • Compliance-ready audit logging")
print("   • Production deployment patterns")

print("\n🚀 Next Steps:")
print("   1. Deploy using these production patterns")
print("   2. Customize configuration for your requirements")
print("   3. Set up monitoring dashboards and alerts")
print("   4. Conduct security reviews and testing")
print("   5. Validate compliance with regulations")

print("\n💡 Key Takeaways:")
print("   • Security is paramount - always use PII detection")
print("   • Monitor everything for proactive issue resolution")
print("   • Plan for failure with robust error handling")
print("   • Build compliance features from the start")
print("   • Use session pooling for efficient scaling")

print("\n🎯 This completes the production LlamaIndex-AgentCore tutorial series!")
print("   You now have enterprise-ready patterns for secure, scalable AI applications.")